In [ ]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import math

out = JupyterOut.unit_square( )

# Point Managers

A number of useful geometries that are no doubt familiar to CAD users - polylines, polygons and meshes - base their computational description upon ***a managed set of Points***. Decod.es offers a structure that encapsulates routines expressly for this purpose: the abstract ***HasPts*** class.

This class extends HasBasis and implements functionality common to ***any geometry that allows access to a collection of Points as construction geometry in relationship to a basis***. 

Implicit in this definition is a separation between local coordinates described relative to the basis, and global coordinates in world space 



<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P27.jpg" style="width: 200px; display: inline;">


## HasPts Objects in Decod.es

The primary responsibility of the HasPts class is to manage a local set of coordinates and to provide the means for their transformation to and from global coordinates. 

For clarity in discussing the two types of coordinates present in every descendant of HasPts, local coordinates will consistently be termed ***vertices*** or ***verts*** and manifest in code as Decod.es Vecs. In contrast, global positions will be referred to as ***points*** or ***pts*** and will manifest as Decod.es Points. By convention, if a method is passed a Point it is assumed to be a description of a location in global coordinates, and if passed a Vec we assume it to be a local position relative to the basis.

The local vertices are regarded as a more essential description which must be evaluated by the stored coordinate system to derive points in global space.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/3.00.D50 HasPts Expanded.jpg" style="width: 200px; display: inline;">


### HasPts Members and Methods

Any class descendant from HasPts is expected to store two members: 
* `hazpts._verts`, a collection of Vecs that describe the vertices of the parent geometry
* `hazpts._basis`, a CS that is employed to evaluate these vertices to produce a collection of derivative Points

A constructor is defined. The positions passed to this constructor, be they Vecs or Points, are expected to describe local coordinates. If no basis is given, a basis of world space is assumed, and local and global coordinates then coincide.

In [ ]:
"""
HasPts Initialization
The abstract HasPts constructor takes a set of vertices in local coordinates 
and a CS basis.
"""
    def __init__(self, verts=None,basis=None):
        self._verts = []
        # append verts before basis to ensure local interpretation
        if verts is not None: self.append(verts)
        self._basis = basis

What is not immediately apparent here is that the `hazpts.append()` method has the effect of converting any given vertices to Vecs before storing them in the `hazpts._verts` member.

While the derivative collection of Points is made accessible via the `hazpts.pts` property, its important to remember that they are the product of the `hazpts.basis.eval()` method of the stored CS, which effectively maps local coordinates to global coordinates.

In [ ]:
"""
HasPts Points
Here, the local vertices contained within this HasPts are transformed into 
global Points by evaluating these coordinates by the stored CS.
"""
@property
def pts(self):
    if self.is_baseless :  return tuple([Point(v) for v in self._verts])
    return tuple([Point(self._basis.eval(v)) for v in self._verts])

TODO: HasPts properties and methods

## PLine Objects in Decod.es

A polyline is understood as a chain of connected line segments strung through a series of Points. 

The PLine class in Decod.es does not require additional members beyond those defined by its forbearer HasPts, but does extend and introduce a number of methods that interpret the managed collection of Points as a chain of connected segments.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/3.00.D56 Pline Large.jpg" style="width: 800px; display: inline;">


Most of the work of interpreting the inherited functionality of a managed collection of Points is handled by the `pl.seg()` method, which returns a Segment that spans a pair of Points associated with a given index.

In [ ]:
"""
Polyline Segment
Returns a Segment of this PLine that is associated with a given index. If a 
negative index or an index larger than the number of Segments is given, the 
index is modified such that the resulting Segments cycle across the length of 
the PLine.
"""
def seg(self,idx):
    idx = idx%(len(self)-1)
    return Segment(self.pts[idx],self.pts[idx+1])

This function is called upon by the pl.edges property, which aggregates each available Segment into a collection of edges.

In [ ]:
"""
Polyline Edges
Returns the all the Segments contained within this PLine.
"""
@property
def edges(self):
    return tuple([ self.seg(n) for n in range(len(self)-1) ])

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P16.jpg" style="width: 200px; display: inline;">


Like the Segments of which they are comprised, PLines may be evaluated. However, the effective parameterization of a PLine is not smooth. Rather, equally spaced values of t will result in a variable spaced set of Points, as determined by the relative distances between the PLine vertices.

In [ ]:
"""
Polyline Evaluation
Evaluates this PLine at the given normalized parameter t.
"""
def eval(self,t):
    if t == 0.0 : return self.pts[0]
    if t == 1.0 : return self.pts[-1]
    for seg,ival in zip(self.edges, Interval()//(len(self)-1)):
        if t in ival: return seg.eval(ival.deval(t))

The `pl.reversed()` method returns a copy of a PLine object with the same basis, but with its vertices ordered in the opposite direction, effectively reversing the direction of the polyline.

In [ ]:
"""
Polyline Reversal
Returns a copy of this PLine with the vertices reversed, an action which 
requires direct access to the private _verts member.
"""
def reversed(self):
    return PLine(reversed(self._verts),self.basis)

# Accessing and Manipulating HasPts Objects
Here we discuss the distinction between local and global positions, and examine how this difference is handled in practice.

We know the distinction of returned types: the private member `hazpts._verts` returns Vecs describing local positions, and the property `hazpts.pts` returns Points describing global locations. How does this convention affect the ways in which coordinate positions are accessed, manipulated, and appended to existing HasPts objects?

Several cases are described below that illustrate these issues through the construction and manipulation of a PLine. While there are many cases for which no intervention or new functionality is required beyond what has already been elaborated, a number of situations will require the development of routines not yet covered.

A note. Square-bracket notation structure is implemented in the HasPts class to offer convenient access to the private `hazpts._verts` collection, such that `hazpts[0]` is an equivalent statement to `hazpts._verts[0]`, and provides access to a local-coordinate Vec of the HasPts object. Slicing is allowed for retrieving vertices, such that `hazpts[2:5]` returns a List of three Vecs.

In the service of some of the cases described below, a method is defined that attempts to cast a given object into a Vec that describes a desired location in local coordinates. 

Three potential scenarios are addressed. 

* If the HasPts object has no basis, then no calculations are required and, no matter the type of object being passed, the coordinates may be used as given. In this case, the given object is converted to a vector and returned. 
* If a Point is given and the HasPts object is based, we interpret the described location as residing in world-space. In this case, it is necessary to call upon the `cs.deval()` method of the basis to calculate the proper local-space coordinates and return the appropriate Vec. 
* If the HasPts object is based and a Vec is given, we interpret the Vec as describing local-space coordinates, and simply return a copy of the given object converted to a Vec.

In [ ]:
"""
HasPts Vector Compatibility
Given either a Point in world space or a Vec in local space, this private 
method returns a location compatible with the basis of this HasPts object. 
This may require devaluating the given coordinate by the basis CS.
"""
def _compatible_vec(self,other):
    # if this object is baseless, return raw coords
    if self.is_baseless:  return Vec(other) 
    # if based, and the other is a Point, devaluate by basis
    if isinstance(other, Point): return Vec(self._basis.deval(other))
    # if based, and other has xyz coords, return raw coords
    return Vec(other)

With a procedure defined that is able to cast any given coordinate object into a vector compatible with the basis of a HasPts, the `hazpts.append()` method is reduced to a largely managerial role. Here, we offer the calling context some flexible argumentation that allows for either a single object or an iterable List of objects to be passed.

In [1]:
"""
HasPts Append
Appends the given Point or collection of Points to the List of vertices 
maintained by this HasPts. Each given coordinate is processed to ensure 
compatibility with this geometry's basis.
"""
def append(self,obj):
    try : 
        for p in obj : 
            self._verts.append(self._compatible_vec(p))
    except : 
        self._verts.append(self._compatible_vec(obj))

The cases articulated below demonstrate the need for both distinguishing between global Points and local Vecs passed to a HasPts method, and, when appropriate, to transform a given set of global coordinates into a set of local coordinates that are compatible with the basis of a HasPts.

### Case 1: PLine Construction by Points Only

Like many HasPts objects, a PLine may be constructed by simply passing a collection of vertices. Since no basis is given, a basis of R3 (equivalent to CS()) is assumed.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P01.jpg" style="width: 200px; display: inline;">

In [ ]:
pl = PLine(pts)

### Case 2: PLine Construction By Basis And Vertices

A PLine may be constructed by a collection of vertices and a CS basis. The given locations, which could be Vecs or Points, are interpreted as local positions defined relative to the given coordinate system. The `pl.basis` shown in the nearby diagram is at `(2,0,0)`.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P02.jpg" style="width: 200px; display: inline;">


In [ ]:
pl = PLine(pts,cs)

### Case 3: Appending Coords To A Baseless PLine

If a PLine has no basis, then local and global coordinates are coincident, and any locations appended (both Points and Vecs) may be added directly to the `pl._verts` collection.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P03.jpg" style="width: 200px; display: inline;">


In [ ]:
pl = PLine(pts)
pl.append(Point(0,0,1.5))

### Case 4: Altering Points Of A Based PLine

The vertices of a PLine that stores a basis may be directly accessed, manipulated, and re-assigned via the `pl._verts` member, or by using square bracket notation as seen below. Only Vecs should be used to define local coordinates when directly manipulating vertices in this way.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P04.jpg" style="width: 200px; display: inline;">


In [ ]:
pl = PLine(pts,cs)
pl[-1] = Vec(0,0,1.5)

### Case 5: Altering The Basis Of A Based PLine

The resulting geometry of any HasBasis object may be transformed via manipulations of its underlying basis. Here, the basis of a PLine is re-defined such that the world-space polyline is accordingly re-oriented.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P05.jpg" style="width: 200px; display: inline;">


In [ ]:
pl = PLine(pts,cs)
pl.basis = CS(Point(2,0),Vec(-1,0))

### Case 6: Appending Coords To A Based PLine

When appending new positions to an existing PLine that stores a basis, we must differentiate between global and local locations. It is assumed that a local location is described if a Vec is given, and a global location if a Point is given.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P06.jpg" style="width: 200px; display: inline;">


In [ ]:
pl = PLine(pts,cs)
pl.append(Vec(0,0,1.5))
pl.append(Point(2,0,2))

# PGon and RGon Objects in Decod.es

Geometrically, a polygon is understood to be a closed planar shape comprised of straight edges. A regular polygon constrains this definition to include only those shapes that exhibit equal-length edges that are non-overlapping. 

While we might understand such entities as special cases of a polyline, in practice their implementation suggests a different structure of inheritance. 

A Decod.es PGon and RGon differ sufficiently from that of a PLine as to suggest a sibling relationship with polylines. However, they are similar enough as to warrant an inheritance relationship with one another. 

As such, ***the PGon class is defined as a special kind of HasPts, and an RGon is defined as a special kind of PGon***.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.C01.jpg" style="width: 800px; display: inline;">


### PGons in Decod.es

While the Decod.es PGon does not introduce new members to the HasPts class, it does add new methods and extend existing ones in order to express two core characteristics of this geometry: 
* planarity
* a closed loop of segments that lie on the cs.xy_plane of its basis

Complications arise in initializing a PGon when conflicting or ambiguous information is provided in regards to the vertices given and the `cs.xy_plane` of the basis on which these vertices lie. 

For example, imagine that we attempt to construct a PGon out of a set of non-planar Points. How should we interpret the intention of the calling statement in this situation? Or, consider that a calling statement attempts to construct a PGon from a set of non-planar Points but offers no defined basis. While we could perform the same operation assuming a basis of R3, another route may prove more savvy. In cases such as this, the Decod.es library attempts to automatically fit a coordinate system to the given set of Points and assigns this as the basis of the resulting PGon.

In [ ]:
"""
PGon Initialization
The PGon constructor takes a defined basis, a collection of vertices, or both. 
If only a collection of verts is given, these are interpreted as world-space 
Points, and an attempt is made to find a Plane in which they lie.
"""
class PGon(HasPts):
    def __init__(self, vertices=None, basis=None):
        if basis is None and vertices is None : raise GeometricError()
        # if vertices have been provided, but no basis:
        if basis is None:
            # find the plane that best fits the given world-space Points
            # define a CS using the first three Points given
            # initialize using the CS and given Points via HasPts constructor
        # if only a basis or a basis and vertices have been provided:
        else:
            #strip the z-coordinate and initialize via HasPts constructor
            super(PGon,self).__init__([Vec(v.x,v.y) for v in vertices],basis)
            # initialize the the basis 
            self.basis = basis 

The enforced planarity of a PGon also requires a nuanced approach to appending new vertices, as these must be checked to ensure they lie on the xy-plane of the existing basis. For this reason, the PGon class extends the `hazpts.append()` method.

In [ ]:
"""
PGon Append
When a new location is added to an existing PGon, each Point or Vec given is 
first processed by the HasPts appending mechanism, and then, after they have
been added to the list of stored _verts, they are processed to ensure planarity.
"""
def append(self,pts):
    super(PGon,self).append(pts)
    try:
        for n in range(len(pts)): self._verts[-(n+1)].z = 0
    except:
        self._verts[-1].z = 0   

Because polygons are closed, when computing features that index the edges of a PGon, we must account for the final Segment that closes this object by joining the last vertex with the first. Note the difference between the `pg.seg()` method below and the similar one defined for polylines, above.

In [ ]:
"""
PGon Segments and Edges
The PGon class constructs Segments and edges differently than a PLine object 
in Decod.es, accounting for the closed nature of a polygon by including the 
last Segment that connects the last vertex to the first.
"""
def seg(self,idx):
    idx_a, idx_b = idx%(len(self)), (idx+1)%(len(self))
    return Segment(self.pts[idx_a],self.pts[idx_b])
        
@property
def edges(self):
    return [self.seg(n) for n in range(len(self))]

A host of methods follow from this basic structure. Some of these require only elementary geometry, while others require more involved routines.

TODO: PGON Properties and Methods

### RGons in Decod.es

A regular polygon can be regarded as a special case of a polygon. By structuring Decod.es RGons to extend Decod.es PGons, we enjoy the significant advantage of certain relevant routines that are provided gratis.

To enjoy the advantage of these inherited methods we must also mitigate the collateral problems. 

In this case, we find ***a discrepancy between the basic assumption of a progenitor class and the requirements of its descendant***. 

The issue is this: ***RGons maintain a collection of vertices, but are not usefully defined by them***, rather, an RGon is better regarded as defined by a more central set of members. More than demonstrate the geometric properties of regular polygons, this section illuminates the ***challenges in navigating the differing assumptions between inherited functionality and the features of a descendant type***.

While a regular polygon may be usefully described by a range of attributes, the Decod.es RGon identifies three as the most central and that are defined as members in its construction. A regular polygon is defined as:
* a radius `rg._radius`
* an Integer number of sides `rg._nos`
* a base coordinate system `rg._basis` on which it is centered

For convenience, either the length of each edge `rg.edge_length` or the apothem `rg.apothem`, which describes the distance from the center to the middle of any one of its edges may be provided via keyword to the RGon constructor in place of the radius.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P17.jpg" style="width: 200px; display: inline;">


Note that none of the core members listed above refer to the vertices of the RGon, that in order to inherit the useful methods of a PGon, our regular polygon must fulfill its responsibilities as a HasPts descendant. This requires the definition of a collection of local vertices `hazpts._verts`, which can be derived using the given information.

In [ ]:
"""
Computing the Vertices of a RGon
To fulfill its responsibilities as a HasPts, a Decod.es RGon must compute a set
of vertices from its core members.
"""
stp = pi*2.0/num_of_sides
crds = [(radius*cos(stp*n), radius*sin(stp*n)) for n in range(num_of_sides)]
verts = [Point(x,y) for x,y in crds]

Consider now the implications of some of the other functionality that RGon inherits from HasPts. 

The HasPts class has gone to great lengths to ensure that new positions may be added to the stored array of vertices in a variety of ways. Yet, because we have chosen to define an RGon as a set of largely numeric members rather than as a managed collection of Points, ***the manual addition of vertices would be nonsensical for an RGon***.

The most Python-ic option is not to forbid this action, but to simply discourage it, in this case overriding HasPts with an `rg.append()` method that does nothing. And so we find ourselves in the odd position of demonstrating the removal of functionality from a class.

In [ ]:
"""
Appending Vertices to an RGon
It is not possible to append vertices to an RGon. 
"""
def append(self,pts):
    raise GeometricError("I can't even. You can't append vertices to a RGon!")

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P18.jpg" style="width: 200px; display: inline;">


In [ ]:
"""
RGon Inflate
Constructs an RGon inscribing the given RGon by specifying the t-value of the 
intersection point.
"""
def inflate(self, t):
    pa = Point.interpolate(self.pts[0],self.pts[1],t)
    o = self._basis.origin
    x,y = Vec(o,pa), self._basis.z_axis.cross(Vec(o,pa))
    r = o.dist(pt_a)
    return RGon(self._nos, basis = CS(o,x,y), radius = r)

In [ ]:
"""
RGon Deflate
Constructs an RGon circumscribing the given RGon by specifying the t-value of 
the perpendicular point.
"""
def deflate(self, t):
    pa = Point.interpolate(self.pts[0],self.pts[1],t)
    pb = Point.interpolate(self.pts[-1],self.pts[0],t)
    o = self._basis.origin
    x,y = Vec(o,pa), self._basis.z_axis.cross(Vec(o,pa))
    vec_perp = (Vec(pb, pa).cross(self._basis.z_axis)).normalized()
    a = Vec(o,self.pts[0]).dot(vec_perp)
    return RGon(self._nos, basis=CS(o,x,y), apothem = a)

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P19.jpg" style="width: 200px; display: inline;">


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P20.jpg" style="width: 200px; display: inline;">
